In [10]:
import nltk
import pandas as pd
import re
from nltk.corpus import stopwords
from textblob import TextBlob
from nltk.tokenize import word_tokenize
from nltk import WordNetLemmatizer
from string import punctuation
import string

In [15]:
data = pd.read_excel('K8 Reviews v0.2.xlsx')

In [16]:
data.head()

,sentiment,review
0,1,Good but need updates and improvements
1,0,"Worst mobile i have bought ever, Battery is dr..."
2,1,when I will get my 10% cash back.... its alrea...
3,1,Good
4,0,The worst phone everThey have changed the last...


In [37]:
def clean_text(sent):
    sent =sent.lower()
    tokens = word_tokenize(sent)
    stop_updated = stopwords.words('english')+list(punctuation)
    final_word = [term for term in tokens if term not in stop_updated and len(term)>2]
    res= ' '.join(final_word)
    return res
    
    

In [19]:
data['review'].dtype

dtype('O')

In [35]:
data['review']= data['review'].astype('str')

In [33]:
#data['review']=data['review'].apply(lambda x: x.lower())
#data.head()

In [34]:
#data['clean'] = data['review'].apply(lambda x: re.sub(r'[^\w\s]','',x))
#data.head()

In [38]:
data['clean'] = data['review'].apply(lambda x:clean_text(x))

In [39]:
data['clean'].head()

0                       good need updates improvements
1    worst mobile bought ever battery draining like...
2                   get cash back .... already january
3                                                 good
4    worst phone everthey changed last phone proble...
Name: clean, dtype: object

In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [42]:
tfidf = TfidfVectorizer(ngram_range=(1,2), max_features=10)

X = tfidf.fit_transform(data['clean'])

dtm_imdb = pd.DataFrame(X.toarray(),columns=tfidf.get_feature_names())
dtm_imdb


,battery,camera,good,lenovo,mobile,nice,phone,problem,product,quality
0,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,0.611600,0.000000,0.000000,0.372605,0.697933,0.000000,0.000000,0.000000,0.000000,0.0
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
3,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.876165,0.482011,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...
14670,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.0
14671,0.533168,0.541936,0.000000,0.649643,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
14672,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
14673,0.000000,0.559134,0.000000,0.000000,0.000000,0.700409,0.443618,0.000000,0.000000,0.0


In [43]:
df= data.copy()

In [44]:
x= df['clean']
y=df['sentiment']

In [45]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.2)

In [46]:
tfidf = TfidfVectorizer(ngram_range=(1,2))

In [47]:
xtrain_bow = tfidf.fit_transform(xtrain)

In [50]:
xtest_bow = tfidf.transform(xtest)

In [48]:
from sklearn.linear_model import LogisticRegression


In [51]:
clf = LogisticRegression().fit(xtrain_bow,ytrain)
pred = clf.predict(xtest_bow)

In [52]:
from sklearn.metrics import classification_report,accuracy_score, confusion_matrix

In [53]:
accuracy_score(ytest , pred)

0.8528109028960817